In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os, sys, time, tarfile, subprocess, textwrap, shutil, datetime
from pathlib import Path

REPO_URL = "https://github.com/SzymonSmagowski/DeepLearningCourse.git"
BRANCH = "main"
DATA_IN_DRIVE = "data/processed_cats.zip"

ROOT_DRIVE = Path("/content/drive/MyDrive")
DATA_SRC = ROOT_DRIVE / DATA_IN_DRIVE

DATA_DST = Path("/content/data")
REPO_DIR = Path("/content") / Path(REPO_URL).stem
OUTPUTS_DIR = ROOT_DRIVE / "project_3" / datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
PROJECT_DIR = REPO_DIR / "project_3"

print("Repo dir:", REPO_DIR)
print("Data src:", DATA_SRC)
print("Data dst:", DATA_DST)
print("Outputs dir:", OUTPUTS_DIR)

In [ ]:
if not REPO_DIR.exists():
    !git clone -b "$BRANCH" "$REPO_URL" "$REPO_DIR"
else:
    %cd $REPO_DIR
    !git pull origin "$BRANCH"
    %cd -

In [ ]:
DATA_NAME   = "processed_cats.zip"

# full path once extracted in Colab
LOCAL_DATA = PROJECT_DIR / 'data'
LOCAL_DATA_ZIP = LOCAL_DATA / DATA_NAME
LOCAL_DATA.mkdir(exist_ok=True, parents=True)

# --- logic --------------------------------------------------
if LOCAL_DATA_ZIP.exists():
    print(f"✓ dataset already present at {LOCAL_DATA_ZIP}")
elif DATA_SRC.exists():                                # extracted on Drive
    LOCAL_DATA_ZIP.symlink_to(DATA_SRC)
    print(f"🔗  Symlinked {DATA_SRC} → {LOCAL_DATA}")
else:
    raise FileNotFoundError(
        "Dataset not found!\n"
        f"Looked for: {DATA_SRC}"
    )

In [ ]:
!unzip -nq $LOCAL_DATA_ZIP -d $LOCAL_DATA

In [ ]:
!pip install -q -r DeepLearningCourse/project_3/requirements.txt

In [ ]:
def run_and_tee(cmd, cwd, extra_env=None):
    """
    Run *cmd* (list/str) inside *cwd*. Return exit-code.
    """
    env = os.environ.copy()
    if extra_env:
        env.update(extra_env)

    if isinstance(cmd, str):
        cmd = cmd.split()

    print("💻", " ".join(cmd))
    proc = subprocess.Popen(
        cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT,
        cwd=str(cwd), env=env, text=True
    )
    for line in proc.stdout:
        print(line, end="")
    proc.wait()
    return proc.returncode

In [ ]:
CONFIGS = [
    ('DIFFUSION', 'diffusion_fast_test.yaml'),
    ('GAN', 'gan_fast_test.yaml')
#     ('DIFFUSION', 'diffusion_128.yaml'),
#     ('DIFFUSION', 'diffusion_256.yaml'),
#     ('GAN', 'gan_128.yaml'),
#     ('GAN', 'gan_256.yaml'),
]

cmd = [
    sys.executable, 'colabify_configs.py',
    '--prefix', str(OUTPUTS_DIR)
]
code = run_and_tee(cmd, cwd=PROJECT_DIR)
if code != 0:
    raise RuntimeError(f"colabify_configs.py failed (exit {code})")

In [ ]:
failed = []
for model, config_name in CONFIGS:
    model_path = PROJECT_DIR / model
    mname      = model_path.stem
    print(f"\n▶️  Running {model} on config {config_name}")
    config_path = PROJECT_DIR / 'configs' / config_name

    executable = PROJECT_DIR / \
     ('train_diffusion.py' if model == 'DIFFUSION' else 'train_gan.py')
    run_results = OUTPUTS_DIR / mname
    cmd = [
        sys.executable, str(executable),
        '--config', str(config_path)
    ]
    rc = run_and_tee(cmd, cwd=PROJECT_DIR)
    if rc == 0:
        print(f"✅  {model} / {config_name} finished OK")
    else:
        print(f"❌  {model} / {config_name} FAILED (exit {rc})")
        failed.append((model, config_name))

print("\n" + "═"*35 + " SUMMARY " + "═"*35)
if failed:
    print("The following runs failed:")
    for m, c in failed:
        print(f"  • {m}: {c}")
else:
    print("🎉  All runs completed successfully!")